In [2]:
# loops thru each first-round NFL playoff matchup
# adds quarterfinals to the EV calculation



import mysql.connector
from collections import defaultdict

# Database connection setup
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conference, Championship, and Quarterfinals events
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship', 'Quarterfinals')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch and organize payouts in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0, 'payout_quarterfinals': 0})

# Matchup structure with actual team names
matchups = {
    "AFC": [
        ("Kansas City Chiefs", "Miami Dolphins"),
        ("Cincinnati Bengals", "Pittsburgh Steelers"),
        ("Jacksonville Jaguars", "Cleveland Browns")
    ],
    "NFC": [
        ("San Francisco 49ers", "Minnesota Vikings"),
        ("Detroit Lions", "Atlanta Falcons"),
        ("Dallas Cowboys", "Seattle Seahawks")
    ]
}

# Define probabilities and payouts for each team
team_probabilities = {
    "Kansas City Chiefs": {'current_round_prob': 0.75, 'current_quarterfinals_prob': 0.3, 'current_conference_prob': 0.2, 'current_champ_prob': 0.1},
    "Miami Dolphins": {'current_round_prob': 0.25, 'current_quarterfinals_prob': 0.12, 'current_conference_prob': 0.1, 'current_champ_prob': 0.05},
    "Cincinnati Bengals": {'current_round_prob': 0.7, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Pittsburgh Steelers": {'current_round_prob': 0.3, 'current_quarterfinals_prob': 0.14, 'current_conference_prob': 0.08, 'current_champ_prob': 0.04},
    "Jacksonville Jaguars": {'current_round_prob': 0.6, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.15, 'current_champ_prob': 0.07},
    "Cleveland Browns": {'current_round_prob': 0.4, 'current_quarterfinals_prob': 0.28, 'current_conference_prob': 0.09, 'current_champ_prob': 0.03},
    "San Francisco 49ers": {'current_round_prob': 0.75, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.22, 'current_champ_prob': 0.12},
    "Minnesota Vikings": {'current_round_prob': 0.25, 'current_quarterfinals_prob': 0.12,'current_conference_prob': 0.11, 'current_champ_prob': 0.05},
    "Detroit Lions": {'current_round_prob': 0.65, 'current_quarterfinals_prob': 0.3,'current_conference_prob': 0.17, 'current_champ_prob': 0.07},
    "Atlanta Falcons": {'current_round_prob': 0.35, 'current_quarterfinals_prob': 0.16,'current_conference_prob': 0.08, 'current_champ_prob': 0.03},
    "Dallas Cowboys": {'current_round_prob': 0.6, 'current_quarterfinals_prob': 0.28,'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Seattle Seahawks": {'current_round_prob': 0.4, 'current_quarterfinals_prob': 0.18,'current_conference_prob': 0.09, 'current_champ_prob': 0.04}
}

# Populate payouts based on the fetched data
for participant_name, event_type, total_payout in cursor.fetchall():
    payouts[participant_name][f'payout_{event_type.lower().replace(" ", "_")}'] = float(total_payout)

cursor.close()
conn.close()

# Define EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_quarterfinals_prob, quarterfinals_payout, current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    conditional_quarterfinals_prob = current_quarterfinals_prob / current_round_prob
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    
    quarterfinals_ev = calculate_ev(conditional_quarterfinals_prob, quarterfinals_payout)
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    
    return quarterfinals_ev + conference_ev + champ_ev

# Loop through each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference First-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1 = {
            "current_round_prob": team_probabilities[team1_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team1_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team1_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team1_name]["payout_quarterfinals"],
            "conference_payout": payouts[team1_name]["payout_conference"],
            "champ_payout": payouts[team1_name]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_name]["current_round_prob"],
            "current_quarterfinals_prob": team_probabilities[team2_name]["current_quarterfinals_prob"],
            "current_conference_prob": team_probabilities[team2_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_name]["current_champ_prob"],
            "quarterfinals_payout": payouts[team2_name]["payout_quarterfinals"],
            "conference_payout": payouts[team2_name]["payout_conference"],
            "champ_payout": payouts[team2_name]["payout_championship"]
        }

        # Calculate conditional EVs
        team1_ev_win = calculate_conditional_ev(
            team1["current_quarterfinals_prob"], team1["quarterfinals_payout"],
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_ev_win = calculate_conditional_ev(
            team2["current_quarterfinals_prob"], team2["quarterfinals_payout"],
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # Calculate the Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        # Output results
        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"Series Delta: ${series_delta:.2f}\n")



AFC Conference First-Round Matchups:

Kansas City Chiefs vs Miami Dolphins
Kansas City Chiefs EV if win round: $14369.33
Miami Dolphins EV if win round: $0.00
Series Delta: $14369.33

Cincinnati Bengals vs Pittsburgh Steelers
Cincinnati Bengals EV if win round: $12003.43
Pittsburgh Steelers EV if win round: $25804.67
Series Delta: $13801.24

Jacksonville Jaguars vs Cleveland Browns
Jacksonville Jaguars EV if win round: $2628.50
Cleveland Browns EV if win round: $0.00
Series Delta: $2628.50


NFC Conference First-Round Matchups:

San Francisco 49ers vs Minnesota Vikings
San Francisco 49ers EV if win round: $0.00
Minnesota Vikings EV if win round: $10595.00
Series Delta: $10595.00

Detroit Lions vs Atlanta Falcons
Detroit Lions EV if win round: $19313.00
Atlanta Falcons EV if win round: $0.00
Series Delta: $19313.00

Dallas Cowboys vs Seattle Seahawks
Dallas Cowboys EV if win round: $4506.67
Seattle Seahawks EV if win round: $6133.50
Series Delta: $1626.83



In [2]:
# Playoffs Round/Series EV Calculator (Isolated to the Matchup)


import mysql.connector
from collections import defaultdict

conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conf, Champ
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch results and organize them in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Define the relevant teams by full names
relevant_teams = [
    
    ## INSERT FULL TEAM NAMES HERE
    'Detroit Tigers', 'Cleveland Guardians'
]

# Fetch and organize the payouts by relevant teams
for participant_name, event_type, total_payout in cursor.fetchall():
    # Check if the participant name matches any relevant team
    if participant_name in relevant_teams:
        team_name = participant_name
        total_payout = float(total_payout)  # Convert to float for compatibility
        
        if event_type == 'Conference Winner':
            payouts[team_name]['payout_conference'] = total_payout
        elif event_type == 'Championship':
            payouts[team_name]['payout_championship'] = total_payout
        
        # Print out the potential payout for verification
        print(f"Team: {team_name}, Event Type: {event_type}, Total Payout: {total_payout}")

cursor.close()
conn.close()

# Define the EV calculation functions
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the two teams
team1 = { 
    
    ## INSERT PROBABILITIES BELOW
    
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": payouts['Detroit Tigers']['payout_conference'],             
    "champ_payout": payouts['Detroit Tigers']['payout_championship']               
}

team2 = { 
    "current_round_prob": 1 - team1["current_round_prob"],      
    
     ## INSERT PROBABILITIES BELOW
    
    "current_conference_prob": 0.15,    
    "current_champ_prob": 0.06,       
    "conference_payout": payouts['Cleveland Guardians']['payout_conference'],  
    "champ_payout": payouts['Cleveland Guardians']['payout_championship']   
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Series Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("Conditional Expected Values if Team Wins Current Round:")
print(f"Detroit Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Cleveland Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Detroit Tigers EV if win round: $0.00
Cleveland Guardians EV if win round: $0.00

Series Delta:
$0.00


In [4]:
## Playoffs Individual Game EV Calculator (Isolated to the Matchup)


def calculate_team_ev(new_series_odds, series_ev):
    """Calculate the team EV based on new series odds and series EV."""
    return new_series_odds * series_ev

def calculate_game3_ev_delta(team1_game3_win_prob, team2_game3_win_prob, team1_new_series_odds, team2_new_series_odds, team1_ev_win, team2_ev_win):
    """Calculate the EV Delta for Game 3."""
    
    # Calculate EV if Team 1 wins Game 3
    team1_ev_if_win = calculate_team_ev(team1_new_series_odds, team1_ev_win)
    team2_ev_if_lose = calculate_team_ev(1 - team1_new_series_odds, team2_ev_win)
    ev_if_team1_wins_game3 = team1_ev_if_win + team2_ev_if_lose
    
    # Calculate EV if Team 2 wins Game 3
    team1_ev_if_lose = calculate_team_ev(1 - team2_new_series_odds, team1_ev_win)
    team2_ev_if_win = calculate_team_ev(team2_new_series_odds, team2_ev_win)
    ev_if_team2_wins_game3 = team1_ev_if_lose + team2_ev_if_win
    
    # Calculate the EV Delta for Game 3
    ev_delta = abs(ev_if_team1_wins_game3 - ev_if_team2_wins_game3)
    
    return ev_if_team1_wins_game3, ev_if_team2_wins_game3, ev_delta

# Inputs specific to Game 3
team1_game3_win_prob = 0.53  # Team 1's probability of winning Game 3
team2_game3_win_prob = (1 - team1_game3_win_prob)  # Team 2's probability of winning Game 3
team1_new_series_odds = 1  # Series odds if Team 1 wins Game 3
team2_new_series_odds = 0.99148936170  # Series odds if Team 2 wins Game 3

# Use team1_ev_win and team2_ev_win directly from the previous script
# Calculate EV if Team 1 or Team 2 wins Game 3 and the EV Delta
ev_if_team1_wins_game3, ev_if_team2_wins_game3, game3_ev_delta = calculate_game3_ev_delta(
    team1_game3_win_prob,
    team2_game3_win_prob,
    team1_new_series_odds,
    team2_new_series_odds,
    team1_ev_win,  # From previous script
    team2_ev_win   # From previous script
)

# Output results
print("Expected Values if either team wins Game 3:")
print(f"EV if Team 1 wins Game 3: ${ev_if_team1_wins_game3:.2f}")
print(f"EV if Team 2 wins Game 3: ${ev_if_team2_wins_game3:.2f}")
print("\nGame 3 EV Delta:")
print(f"EV Delta: ${game3_ev_delta:.2f}")


Expected Values if either team wins Game 3:
EV if Team 1 wins Game 3: $22972.13
EV if Team 2 wins Game 3: $7747.85

Game 3 EV Delta:
EV Delta: $15224.29


In [16]:
def calculate_ev(probability, payout):
    """Calculate expected value based on probability and payout."""
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    """Calculate conditional EV if the team wins the current round."""
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Inputs for the Tigers and Guardians
team1 = {  # Tigers
    "current_round_prob": 0.534,      
    "current_conference_prob": 0.18,      
    "current_champ_prob": 0.06,        
    "conference_payout": 0,             
    "champ_payout": 204452               
}

team2 = {  # Guardians
    "current_round_prob": 1 - team1["current_round_prob"],       
    "current_conference_prob": 0.15,      
    "current_champ_prob": 0.06,         
    "conference_payout": 0,     
    "champ_payout": 59160               
}

# Calculate conditional EVs if each team wins the current round
team1_ev_win = calculate_conditional_ev(
    team1["current_conference_prob"], team1["conference_payout"],
    team1["current_champ_prob"], team1["champ_payout"],
    team1["current_round_prob"]
)

team2_ev_win = calculate_conditional_ev(
    team2["current_conference_prob"], team2["conference_payout"],
    team2["current_champ_prob"], team2["champ_payout"],
    team2["current_round_prob"]
)

# Calculate the Round Delta as the difference between conditional EVs
series_delta = abs(team1_ev_win - team2_ev_win)

# Output results
print("Conditional Expected Values if Team Wins Current Round:")
print(f"Tigers EV if win round: ${team1_ev_win:.2f}")
print(f"Guardians EV if win round: ${team2_ev_win:.2f}")
print("\nSeries Delta:")
print(f"Series Delta: ${series_delta:.2f}")


Conditional Expected Values if Team Wins Current Round:
Tigers EV if win round: $22972.13
Guardians EV if win round: $7617.17

Series Delta:
Series Delta: $15354.97


In [6]:
import mysql.connector
from collections import defaultdict

# Database connection setup
conn = mysql.connector.connect(
    host='betting-db.cp86ssaw6cm7.us-east-1.rds.amazonaws.com',
    user='admin',
    password='7nRB1i2&A-K>',
    database='betting_db'
)

cursor = conn.cursor()

# Query to get Potential Payouts for Conference and Championship events
query = """
    SELECT 
        legs.ParticipantName,
        legs.EventType,
        SUM(bets.PotentialPayout) AS total_payout
    FROM bets
    JOIN legs ON bets.WagerID = legs.WagerID
    WHERE 
        bets.LegCount = 1
        AND bets.WLCA = 'Active'
        AND bets.WhichBankroll = 'GreenAleph'
        AND legs.EventType IN ('Conference Winner', 'Championship')
    GROUP BY legs.ParticipantName, legs.EventType;
"""

cursor.execute(query)

# Fetch and organize payouts in a dictionary
payouts = defaultdict(lambda: {'payout_conference': 0, 'payout_championship': 0})

# Matchup structure with actual team names
matchups = {
    "AFC": [
        ("Kansas City Chiefs", "Miami Dolphins"),
        ("Cincinnati Bengals", "Pittsburgh Steelers"),
        ("Jacksonville Jaguars", "Cleveland Browns")
    ],
    "NFC": [
        ("San Francisco 49ers", "Minnesota Vikings"),
        ("Detroit Lions", "Atlanta Falcons"),
        ("Dallas Cowboys", "Seattle Seahawks")
    ]
}

# Define probabilities and payouts for each team
team_probabilities = {
    "Kansas City Chiefs": {'current_round_prob': 0.75, 'current_conference_prob': 0.2, 'current_champ_prob': 0.1},
    "Miami Dolphins": {'current_round_prob': 0.25, 'current_conference_prob': 0.1, 'current_champ_prob': 0.05},
    "Cincinnati Bengals": {'current_round_prob': 0.7, 'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Pittsburgh Steelers": {'current_round_prob': 0.3, 'current_conference_prob': 0.08, 'current_champ_prob': 0.04},
    "Jacksonville Jaguars": {'current_round_prob': 0.6, 'current_conference_prob': 0.15, 'current_champ_prob': 0.07},
    "Cleveland Browns": {'current_round_prob': 0.4, 'current_conference_prob': 0.09, 'current_champ_prob': 0.03},
    "San Francisco 49ers": {'current_round_prob': 0.75, 'current_conference_prob': 0.22, 'current_champ_prob': 0.12},
    "Minnesota Vikings": {'current_round_prob': 0.25, 'current_conference_prob': 0.11, 'current_champ_prob': 0.05},
    "Detroit Lions": {'current_round_prob': 0.65, 'current_conference_prob': 0.17, 'current_champ_prob': 0.07},
    "Atlanta Falcons": {'current_round_prob': 0.35, 'current_conference_prob': 0.08, 'current_champ_prob': 0.03},
    "Dallas Cowboys": {'current_round_prob': 0.6, 'current_conference_prob': 0.18, 'current_champ_prob': 0.08},
    "Seattle Seahawks": {'current_round_prob': 0.4, 'current_conference_prob': 0.09, 'current_champ_prob': 0.04}
}

# Populate payouts based on the fetched data
for participant_name, event_type, total_payout in cursor.fetchall():
    payouts[participant_name][f'payout_{event_type.lower().replace(" ", "_")}'] = float(total_payout)

cursor.close()
conn.close()

# Define EV calculation functions
def calculate_ev(probability, payout):
    return probability * payout

def calculate_conditional_ev(current_conference_prob, conference_payout, current_champ_prob, champ_payout, current_round_prob):
    conditional_conference_prob = current_conference_prob / current_round_prob
    conditional_champ_prob = current_champ_prob / current_round_prob
    conference_ev = calculate_ev(conditional_conference_prob, conference_payout)
    champ_ev = calculate_ev(conditional_champ_prob, champ_payout)
    return conference_ev + champ_ev

# Loop through each conference and matchup, calculating EVs
for conference, games in matchups.items():
    print(f"\n{conference} Conference First-Round Matchups:\n")
    for team1_name, team2_name in games:
        team1 = {
            "current_round_prob": team_probabilities[team1_name]["current_round_prob"],
            "current_conference_prob": team_probabilities[team1_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team1_name]["current_champ_prob"],
            "conference_payout": payouts[team1_name]["payout_conference"],
            "champ_payout": payouts[team1_name]["payout_championship"]
        }

        team2 = {
            "current_round_prob": team_probabilities[team2_name]["current_round_prob"],
            "current_conference_prob": team_probabilities[team2_name]["current_conference_prob"],
            "current_champ_prob": team_probabilities[team2_name]["current_champ_prob"],
            "conference_payout": payouts[team2_name]["payout_conference"],
            "champ_payout": payouts[team2_name]["payout_championship"]
        }

        # Calculate conditional EVs
        team1_ev_win = calculate_conditional_ev(
            team1["current_conference_prob"], team1["conference_payout"],
            team1["current_champ_prob"], team1["champ_payout"],
            team1["current_round_prob"]
        )

        team2_ev_win = calculate_conditional_ev(
            team2["current_conference_prob"], team2["conference_payout"],
            team2["current_champ_prob"], team2["champ_payout"],
            team2["current_round_prob"]
        )

        # Calculate the Series Delta
        series_delta = abs(team1_ev_win - team2_ev_win)

        # Output results
        print(f"{team1_name} vs {team2_name}")
        print(f"{team1_name} EV if win round: ${team1_ev_win:.2f}")
        print(f"{team2_name} EV if win round: ${team2_ev_win:.2f}")
        print(f"Series Delta: ${series_delta:.2f}\n")



AFC Conference First-Round Matchups:

Kansas City Chiefs vs Miami Dolphins
Kansas City Chiefs EV if win round: $14369.33
Miami Dolphins EV if win round: $0.00
Series Delta: $14369.33

Cincinnati Bengals vs Pittsburgh Steelers
Cincinnati Bengals EV if win round: $8564.57
Pittsburgh Steelers EV if win round: $25804.67
Series Delta: $17240.10

Jacksonville Jaguars vs Cleveland Browns
Jacksonville Jaguars EV if win round: $2628.50
Cleveland Browns EV if win round: $0.00
Series Delta: $2628.50


NFC Conference First-Round Matchups:

San Francisco 49ers vs Minnesota Vikings
San Francisco 49ers EV if win round: $0.00
Minnesota Vikings EV if win round: $10595.00
Series Delta: $10595.00

Detroit Lions vs Atlanta Falcons
Detroit Lions EV if win round: $19313.00
Atlanta Falcons EV if win round: $0.00
Series Delta: $19313.00

Dallas Cowboys vs Seattle Seahawks
Dallas Cowboys EV if win round: $4506.67
Seattle Seahawks EV if win round: $6133.50
Series Delta: $1626.83

